In [57]:
import fitz as PyMuPDF
import json
import os
import psycopg2
from pathlib import Path
from datetime import datetime
import re
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import re
import time

In [14]:
load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_PORT = os.getenv("DB_PORT")
DB_CONNECTION = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
PDF_FOLDER = Path.home() / "Downloads" / "domaindata" # Update this path
METADATA_FILE = Path.home() / "Downloads" / "domaindata" / "metadata.jsonl"
MODEL_NAME = 'all-MiniLM-L6-v2'  # Good balance of speed and quality
BATCH_SIZE = 64  # Increase if you have more RAM

In [38]:
# Create the table. We'll add the vector column later because it depends on model embedding dimension
conn = psycopg2.connect(DB_CONNECTION)
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS document_chunks (
        id SERIAL PRIMARY KEY,
        uuid UUID,
        title TEXT,
        content TEXT,
        chunk_index INTEGER,
        page_numbers TEXT,
        industries TEXT[],
        date TIMESTAMP,
        country_codes TEXT[],
        chunk_length INTEGER,
        created_at TIMESTAMP DEFAULT NOW()
    );""")
    
conn.commit()
cur.close()
conn.close()
print("✓ Database table created/verified")

✓ Database table created/verified


In [39]:
def extract_pdf_text(pdf_path):
    """Extract text from PDF file with page tracking"""
    try:
        doc = PyMuPDF.open(pdf_path)
        pages_data = []
        
        for page_num, page in enumerate(doc):
            page_text = page.get_text().strip()
            if page_text:  # Only keep non-empty pages
                pages_data.append({
                    'page_num': page_num + 1,
                    'text': page_text
                })
        
        doc.close()
        return pages_data
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        return None

In [51]:
import re

def _clean_pdf_text(txt: str) -> str:
    # Minimal cleanup: remove soft hyphens, fix hyphenated line breaks, normalize spaces
    txt = txt.replace('\u00ad', '')                                # soft hyphen
    txt = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', txt)               # hyphenated line breaks
    txt = txt.replace('\r', ' ').replace('\n', ' ')                # newlines -> space
    txt = re.sub(r'\s+', ' ', txt).strip()                         # collapse spaces
    return txt


# Helper to find a nice end near target
def pick_end(lo, tgt, hi, text):
    """Find a good cut position in text between lo..hi, aiming near tgt."""
    sent_re = re.compile(r'[\.!\?…][\"»’”\)\]]*\s')
    # forward first
    m = sent_re.search(text, tgt, hi)
    if m: return m.end()
    # backward
    last = -1
    for m in sent_re.finditer(text, lo, tgt):
        last = m.end()
    if last != -1: return last
    # whitespace
    ws_back = text.rfind(' ', lo, tgt)
    if ws_back != -1: return ws_back + 1
    ws_fwd = text.find(' ', tgt, hi)
    if ws_fwd != -1: return ws_fwd + 1
    return tgt

    
def chunk_text(pages_data, chunk_size=1200, overlap=180, min_chunk=80):
    # 1) Build full_text and track page spans (char ranges)
    full_text = ""
    page_positions = {}
    cur = 0
    for p in pages_data:
        t = _clean_pdf_text(str(p.get("text", "")))
        if not t:
            continue
        wrapped = f" {t} "
        start = cur
        full_text += wrapped
        cur += len(wrapped)
        page_positions[int(p["page_num"])] = (start, cur)

    if not full_text.strip():
        return []

    n = len(full_text)
    chunks, idx = [], 0
    start = 0

    while start < n:
        tgt = min(start + chunk_size, n)
        lo  = max(start + max(0, chunk_size - 200), start)  # look ~200 chars back
        hi  = min(tgt + 200, n)                             # and ~200 ahead
        end = pick_end(lo, tgt, hi, full_text)

        piece = full_text[start:end].strip()
        if end < n and len(piece) < min_chunk:
            # try to extend a bit to avoid tiny fragments
            hi2 = min(end + (min_chunk - len(piece)) + 120, n)
            end = pick_end(end, end, hi2)
            piece = full_text[start:end].strip()

        if piece:
            # pages spanned by this chunk
            chunk_pages = [str(p) for p, (ps, pe) in page_positions.items()
                           if not (end <= ps or start >= pe)]
            chunks.append({
                "text": piece,
                "chunk_index": idx,
                "pages": ",".join(chunk_pages),
                "length": len(piece)
            })
            idx += 1

        if end >= n:
            break

        # next start with overlap, but nudge to next word boundary
        raw_next = max(0, end - overlap)
        # if we’re in the middle of a word, move forward to the next space
        j = raw_next
        while j < n and not full_text[j].isspace():
            j += 1
        # skip consecutive spaces
        while j < n and full_text[j].isspace():
            j += 1
        start = min(j, n)

    return chunks


In [52]:
metadata = {}
missing_count = 0
with open(METADATA_FILE, 'r') as f:
    print("Loading metadata...")
    for line_num, line in enumerate(f, 1):
        if line.strip():
            try:
                data = json.loads(line)
                metadata[data['uuid']] = data
            except json.JSONDecodeError as e:
                print(f"JSON error on line {line_num}: {e}")
    print(f"✓ Loaded metadata for {len(metadata)} documents")

Loading metadata...
✓ Loaded metadata for 500 documents


In [53]:
pdf_files = list(Path(PDF_FOLDER).glob("*.pdf"))
print(f"✓ Found {len(pdf_files)} PDF files")

processed = 0
failed = 0
total_chunks = 0

# Connect to database
conn = psycopg2.connect(DB_CONNECTION)
cur = conn.cursor()

for pdf_file in pdf_files:
    try:
        # Extract UUID from filename
        file_uuid = pdf_file.stem
        
        # Get metadata if available, otherwise use defaults
        if file_uuid in metadata:
            meta = metadata[file_uuid]
            title = meta.get('title', pdf_file.name)
            industries = meta.get('industries', [])
            date = meta.get('date')
            country_codes = meta.get('country_codes', [])
        else:
            # Use filename as fallback
            meta = {}
            title = pdf_file.name
            industries = []
            date = None
            country_codes = []
            no_metadata_count += 1
        
        # Extract text
        pages_data = extract_pdf_text(pdf_file)
        if not pages_data:
            failed += 1
            continue
        
        # Create chunks
        chunks = chunk_text(pages_data)
        
        # Store chunks in database
        for chunk in chunks:
            cur.execute("""
                INSERT INTO document_chunks 
                (uuid, title, content, chunk_index, page_numbers, 
                 industries, date, country_codes, chunk_length)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                file_uuid,
                meta.get('title', ''),
                chunk['text'],
                chunk['chunk_index'],
                chunk['pages'],
                meta.get('industries', []),
                meta.get('date'),
                meta.get('country_codes', []),
                chunk['length']
            ))
        
        total_chunks += len(chunks)
        processed += 1
        
        if processed % 100 == 0:
            conn.commit()
            print(f"  Processed {processed} files, {total_chunks} chunks so far...")
        
    except Exception as e:
        print(f"✗ Error processing {pdf_file.name}: {e}")
        failed += 1

# Final commit
conn.commit()
cur.close()
conn.close()

print(f"\n{'='*60}")
print(f"Processing complete!")
print(f"✓ Successfully processed: {processed} files")
print(f"✗ Failed: {failed} files")
print(f"📄 Total chunks created: {total_chunks}")
print(f"📊 Average chunks per file: {total_chunks/processed if processed > 0 else 0:.1f}")

✓ Found 499 PDF files
Processing: Topcon Investor presentation...
Processing: Gross Domestic Product by State and Personal Incom...
Processing: Irisity Analyst report...
Processing: Saudi Cement Sector Monthly Report October 2024...
Processing: Country Analysis Brief Egypt...
Processing: Empowering Women Through Solar Energy A Case Study...
Processing: Inflation Updated Through August 2024 Regional...
Processing: JGC Investor presentation...
Processing: Ukraine From War to Peace and Recovery...
Processing: Seasonal Summary Crop Report...
Processing: Energy Private Limited Company report...
Processing: Türkiye A Mild Recession as of 3Q24...
Processing: Rhodium Climate Outlook ETSAP Workshop...
Processing: Latin America Corporate Payment Survey 2024...
Processing: Peyto Exploration & Development Analyst report...
Processing: Bridging the Gap Helping Acquirers Meet Evolving M...
Processing: Johnson Controls Quarterly report...
Processing: SunStream Quarterly report...
Processing: BHP Is L

In [54]:
model = SentenceTransformer(MODEL_NAME)
embedding_dim = model.get_sentence_embedding_dimension()
print(f"✅ Model loaded! Embedding dimension: {embedding_dim}")

✅ Model loaded! Embedding dimension: 384


In [55]:
# Add vector column to the database 
conn = psycopg2.connect(DB_CONNECTION)
cur = conn.cursor()

try:
    # Add embedding column
    cur.execute(f"""
        ALTER TABLE document_chunks 
        ADD COLUMN IF NOT EXISTS embedding vector({embedding_dim})
    """)
    print(f"✅ Added embedding column (dimension: {embedding_dim})")
    
    # Create index for faster similarity search
    cur.execute("""
        CREATE INDEX IF NOT EXISTS embedding_cosine_idx 
        ON document_chunks USING ivfflat (embedding vector_cosine_ops) 
        WITH (lists = 20)
    """)
    print("✅ Created vector similarity index")
    
    conn.commit()
    
except Exception as e:
    print(f"Note: {e}")
    # Index creation might fail on empty table, that's OK
    
finally:
    cur.close()
    conn.close()

✅ Added embedding column (dimension: 384)
✅ Created vector similarity index


In [56]:
def check_status():
    """Check how many chunks need embeddings"""
    conn = psycopg2.connect(DB_CONNECTION)
    cur = conn.cursor()
    
    cur.execute("SELECT COUNT(*) FROM document_chunks")
    total_chunks = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM document_chunks WHERE embedding IS NOT NULL")
    embedded_chunks = cur.fetchone()[0]
    
    remaining = total_chunks - embedded_chunks
    
    cur.close()
    conn.close()
    
    print(f"📊 Status:")
    print(f"   Total chunks: {total_chunks:,}")
    print(f"   Already embedded: {embedded_chunks:,}")
    print(f"   Remaining: {remaining:,}")
    
    return total_chunks, embedded_chunks, remaining

total_chunks, embedded_chunks, remaining_chunks = check_status()

📊 Status:
   Total chunks: 12,452
   Already embedded: 0
   Remaining: 12,452


In [58]:
# Generating embeddings for all chunks. This take about 10 minutes on an average CPU with the sleep option.
print(f"🔄 Generating embeddings for {remaining_chunks:,} chunks...")

conn = psycopg2.connect(DB_CONNECTION)
cur = conn.cursor()
processed = 0

while True:
    # Get next batch of chunks without embeddings
    cur.execute("""
        SELECT id, content FROM document_chunks 
        WHERE embedding IS NULL 
        ORDER BY id 
        LIMIT %s
    """, (BATCH_SIZE,))
    
    batch = cur.fetchall()
    if not batch:
        break
    
    # Extract texts and IDs
    chunk_ids = [row[0] for row in batch]
    texts = [row[1] for row in batch]
    
    # Generate embeddings
    try:
        embeddings = model.encode(texts, show_progress_bar=False, convert_to_numpy=True)
        
        # Update database
        for chunk_id, embedding in zip(chunk_ids, embeddings):
            # Convert numpy array to list for postgres
            embedding_list = embedding.tolist()
            cur.execute("""
                UPDATE document_chunks 
                SET embedding = %s 
                WHERE id = %s
            """, (embedding_list, chunk_id))
        
        conn.commit()
        processed += len(batch)
        if processed % 100 == 0:  # Print every 1600 chunks
            print(f"Processed {processed}/{remaining_chunks} chunks ({processed/remaining_chunks*100:.1f}%)")

        time.sleep(1) # Optional to cool my CPU
        
    except Exception as e:
        print(f"❌ Error processing batch: {e}")
        conn.rollback()
        break

cur.close()
conn.close()

print(f"✅ Generated embeddings for {processed:,} chunks!")

# Check final status
print("🔍 Final status check:")
total_chunks, embedded_chunks, remaining_chunks = check_status()

if remaining_chunks == 0:
    print("🎉 All chunks successfully embedded!")
else:
    print(f"⚠️  {remaining_chunks} chunks still need embeddings")

🔄 Generating embeddings for 12,452 chunks...
Processed 1600/12452 chunks (12.8%)
Processed 3200/12452 chunks (25.7%)
Processed 4800/12452 chunks (38.5%)
Processed 6400/12452 chunks (51.4%)
Processed 8000/12452 chunks (64.2%)
Processed 9600/12452 chunks (77.1%)
Processed 11200/12452 chunks (89.9%)
✅ Generated embeddings for 12,452 chunks!
🔍 Final status check:
📊 Status:
   Total chunks: 12,452
   Already embedded: 12,452
   Remaining: 0
🎉 All chunks successfully embedded!


In [68]:
def test_similarity_search(query, top_k=5):
    """Test the similarity search functionality"""
    
    # Generate query embedding
    query_embedding = model.encode([query])[0].tolist()
    
    conn = psycopg2.connect(DB_CONNECTION)
    cur = conn.cursor()
    
    # FIXED: Cast the array to vector type
    cur.execute("""
        SELECT 
            title,
            content,
            1 - (embedding <=> %s::vector) as similarity,
            industries,
            country_codes,
            date::date,
            chunk_index
        FROM document_chunks 
        WHERE embedding IS NOT NULL
        ORDER BY embedding <=> %s::vector
        LIMIT %s
    """, (query_embedding, query_embedding, top_k))
    
    results = cur.fetchall()
    
    # Display results (rest stays the same)
    for i, (title, content, similarity, industries, countries, date, chunk_idx) in enumerate(results, 1):
        print(f"\n{i}. {title} (Chunk {chunk_idx})")
        print(f"   📊 Similarity: {similarity:.3f}")
        print(f"   📅 Date: {date or 'N/A'}")
        print(f"   🏭 Industries: {industries or 'N/A'}")
        print(f"   🌍 Countries: {countries or 'N/A'}")
        print(f"   📄 Content: {content}...")
        print("-" * 60)
    
    cur.close()
    conn.close()
    
    return results

# Test with various queries. I just opened a PDF, saw it refers to AI in banking and insurance 
test_queries = [
    "credit mutuel"
]

for query in test_queries:
    results = test_similarity_search(query, top_k=3)
    print("\n" + "="*100 + "\n")


1. Crédit Mutuel Company report (Chunk 22)
   📊 Similarity: 0.682
   📅 Date: 2025-03-04
   🏭 Industries: ['Finance and Insurance']
   🌍 Countries: ['FR']
   📄 Content: this ranking of the top 500 employers in France, Crédit Mutuel topped the list for the fifth consecutive year, illustrating its performance and proactive human resources policy. The award demonstrates once again that “Working in a bank without shareholders changes everything”! Survey conducted online, independently and anonymously, by the Statista institute among 20,000 employees of 2,100 employers with more than 5,000 employees, between 09/20/2023 and 10/17/2023.  15 ABOUT CRÉDIT MUTUEL GROUP Crédit Mutuel is a mutualist and cooperative banking group exclusively owned by its customermembers. It comprises 18 regional Crédit Mutuel federations and a national agricultural federation as well as numerous specialized subsidiaries, mainly offering financial and insurance services. Its main subsidiaries are CIC, Assurances du 